### Create all preprocessing steps for yolov8 here:
### To train our yolov8 model, we need to make a config.yaml, which needs to contain paths for the train and valid dataset images (we don't have a test, that's on kaggle!). Train should be: datasets/train/images, and datasets/train/labels, Valid should be: datasets/val/images, and datasets/val/labels!

In [4]:
# import necessary packages:
import numpy as np
import os
import torch
import pandas as pd
import json
from torch.utils.data import Dataset
import cv2
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
import shutil
import ast

In [5]:
final_df_src = r"\\fatherserverdw\Kevin\hubmap\obj_detect_bvonly.xlsx"
final_df = pd.read_excel(final_df_src)
final_df

,Unnamed: 0,id,type,coordinates,mask,rle,source_wsi,i,j
0,0,0033bbc76b6b,blood_vessel,"[[[169, 228], [168, 228], [167, 228], [166, 228], [165, 228], [164, 228], [163, 228], [163, 227], [162, 227], [161, 227], [161, 226], [160, 226], [160, 225], [159, 225], [159, 224], [158, 224], [1...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n ...\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]],['73387 16 73897 28 74401 38 74911 43 75420 49 75931 52 76441 55 76952 57 77462 59 77973 62 78481 68 78991 71 79502 73 80013 78 80524 82 81036 86 81548 87 82059 89 82571 90 83083 90 83595 91 84107...,1,10240,43008
1,1,0033bbc76b6b,blood_vessel,"[[[1, 59], [0, 59], [0, 58], [0, 57], [0, 56], [0, 55], [0, 54], [0, 53], [0, 52], [0, 51], [0, 50], [0, 49], [0, 48], [0, 47], [0, 46], [0, 45], [0, 44], [0, 43], [0, 42], [0, 41], [0, 40], [0, 3...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n ...\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]],['30 31 541 32 1053 31 1566 27 2079 25 2594 21 3109 15'],1,10240,43008
2,2,0033bbc76b6b,blood_vessel,"[[[406, 511], [405, 511], [404, 511], [403, 511], [402, 511], [401, 511], [400, 511], [399, 511], [398, 511], [397, 511], [396, 511], [395, 511], [394, 511], [393, 511], [392, 511], [391, 511], [3...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n ...\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]],['190976 1 191485 4 191995 6 192506 7 193017 8 193528 9 194039 10 194551 10 195062 11 195574 11 196086 11 196597 12 197108 13 197620 13 198131 14 198642 15 199154 15 199665 16 200177 16 200689 16 ...,1,10240,43008
3,3,00656c6f2690,blood_vessel,"[[[511, 426], [511, 426], [510, 426], [510, 425], [509, 425], [509, 424], [508, 424], [508, 423], [507, 423], [507, 422], [506, 422], [506, 421], [505, 421], [505, 420], [505, 419], [504, 419], [5...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n ...\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]],['257947 6 258457 12 258967 16 259478 18 259989 20 260500 22 261011 24 261522 26 262033 27'],1,10240,46080
4,4,00656c6f2690,blood_vessel,"[[[157, 404], [156, 404], [155, 404], [154, 404], [153, 404], [152, 404], [151, 404], [150, 404], [149, 404], [148, 404], [147, 404], [146, 404], [145, 404], [145, 403], [144, 403], [143, 403], [1...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n ...\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]],['58240 8 58750 13 59261 15 59772 16 60284 17 60795 18 61306 20 61818 20 62329 21 62841 21 63352 22 63864 22 64376 22 64887 23 65399 24 65911 24 66422 25 66934 26 67446 26 67958 27 68470 27 68982 ...,1,10240,46080
...,...,...,...,...,...,...,...,...,...
3493,3493,fd2437954fd8,blood_vessel,"[[[481, 454], [480, 454], [479, 454], [478, 454], [477, 454], [476, 454], [475, 454], [474, 454], [473, 454], [473, 453], [472, 453], [471, 453], [471, 452], [470, 452], [469, 452], [469, 451], [4...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n ...\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]],['238002 14 238510 20 239012 32 239520 37 240009 60 240515 67 241022 72 241532 75 242023 96 242533 99 243043 101 243541 115 244047 121 244551 129 245056 136 245558 146 246065 151 246575 153 247081...,1,5120,39424
3494,3494,fd2437954fd8,blood_vessel,"[[[416, 511], [415, 511], [414, 511], [413, 511], [412, 511], [411, 511], [410, 511], [409, 511], [408, 511], [407, 511], [406, 511], [405, 511], [404, 511], [403, 511], [402, 511], [401, 511], [4...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n ...\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]],['191998 3 192502 11 193012 13 193522 15 194032 17 194542 19 195051 22 195561 24 196072 25 196584 25 197095 26 197606 27 198116 29 198625 32 199135 34 199645 36 200155 38 200665 40 201173 44 20167...,1,5120,39424
3495,3495,fd2437954fd8,blood_vessel,"[[[18, 362], [17, 362], [16, 362], [16, 361], [15, 361], [14, 361], [13, 361], [13, 360], [12, 360], [1

In [6]:
for idx, row in final_df.iterrows():
    coord = row["coordinates"]
    new_coord = ast.literal_eval(coord)
    final_df.at[idx, "coordinates"] = new_coord

In [7]:
yaml_img_df = final_df.copy()
yaml_img_df = yaml_img_df.groupby('id').agg({'coordinates': list, 'source_wsi': 'first', 'rle': list, 'i' : 'first', 'j' : 'first'}).reset_index()
yaml_img_df

,id,coordinates,source_wsi,rle,i,j
0,0033bbc76b6b,"[[[[169, 228], [168, 228], [167, 228], [166, 228], [165, 228], [164, 228], [163, 228], [163, 227], [162, 227], [161, 227], [161, 226], [160, 226], [160, 225], [159, 225], [159, 224], [158, 224], [...",1,[['73387 16 73897 28 74401 38 74911 43 75420 49 75931 52 76441 55 76952 57 77462 59 77973 62 78481 68 78991 71 79502 73 80013 78 80524 82 81036 86 81548 87 82059 89 82571 90 83083 90 83595 91 8410...,10240,43008
1,00656c6f2690,"[[[[511, 426], [511, 426], [510, 426], [510, 425], [509, 425], [509, 424], [508, 424], [508, 423], [507, 423], [507, 422], [506, 422], [506, 421], [505, 421], [505, 420], [505, 419], [504, 419], [...",1,"[['257947 6 258457 12 258967 16 259478 18 259989 20 260500 22 261011 24 261522 26 262033 27'], ['58240 8 58750 13 59261 15 59772 16 60284 17 60795 18 61306 20 61818 20 62329 21 62841 21 63352 22 6...",10240,46080
2,0067d5ad2250,"[[[[318, 511], [317, 511], [316, 511], [315, 511], [314, 511], [313, 511], [312, 511], [311, 511], [310, 511], [309, 511], [308, 511], [307, 511], [306, 511], [305, 511], [304, 511], [303, 511], [...",2,[['125952 1 126463 2 126973 4 127484 5 127995 6 128506 7 129017 8 129528 9 130038 11 130549 12 131059 14 131570 15 132081 16 132592 17 133102 19 133613 20 134124 21 134635 22 135146 23 135657 24 1...,23552,22528
3,00d75ad65de3,"[[[[385, 393], [384, 393], [383, 393], [382, 393], [381, 393], [380, 393], [379, 393], [378, 393], [378, 392], [377, 392], [376, 392], [376, 391], [375, 391], [375, 390], [374, 390], [373, 390], [...",1,[['178854 41 178944 30 179358 58 179423 82 179862 164 180367 181 180863 199 181367 211 181869 225 182376 232 182885 240 183392 248 183899 256 184407 262 184914 273 185423 301 185932 306 186441 311...,8192,39424
4,00da70813521,"[[[[375, 91], [374, 91], [373, 91], [372, 91], [371, 91], [370, 91], [369, 91], [368, 91], [367, 91], [366, 91], [365, 91], [364, 91], [363, 91], [362, 91], [361, 91], [361, 90], [360, 90], [359, ...",1,[['172618 6 173128 10 173639 12 174150 14 174661 15 175173 15 175685 16 176196 17 176707 19 177219 20 177731 20 178242 22 178754 22 179265 24 179777 24 180288 26 180800 27 181311 28 181823 28 1823...,10240,46592
...,...,...,...,...,...,...
411,f86347534ec1,"[[[[421, 248], [420, 248], [419, 248], [418, 248], [417, 248], [416, 248], [415, 248], [414, 248], [414, 247], [413, 247], [412, 247], [411, 247], [411, 246], [410, 246], [410, 245], [409, 245], [...",2,[['198880 5 199391 8 199902 10 200414 10 200926 10 201438 11 201950 11 202462 12 202974 15 203487 15 203999 16 204511 17 205023 17 205535 17 206047 18 206559 19 207071 19 207583 20 208095 21 20860...,16896,20992
412,faba1bf818ae,"[[[[341, 157], [340, 157], [339, 157], [338, 157], [337, 157], [336, 157], [335, 157], [334, 157], [333, 157], [332, 157], [331, 157], [330, 157], [330, 156], [329, 156], [328, 156], [328, 155], [...",1,[['152181 19 152690 24 153199 28 153699 41 154140 37 154201 52 154650 49 154706 60 155161 117 155672 119 156183 120 156694 121 157204 123 157714 125 158223 128 158733 130 159240 136 159749 139 160...,3072,39424
413,fc6def641612,"[[[[66, 511], [65, 511], [64, 511], [63, 511], [62, 511], [61, 511], [60, 511], [59, 511], [58, 511], [57, 511], [56, 511], [55, 511], [54, 511], [53, 511], [52, 511], [51, 511], [50, 511], [49, 5...",1,[['9727 2 10236 5 10744 9 11252 13 11761 16 12271 18 12779 22 13287 26 13796 29 14306 31 14814 35 15325 36 15835 38 16345 40 16855 42 17366 43 17876 45 18384 49 18893 52 19402 55 19912 57 20422 59...,7680,40960
414,fd2437954fd8,"[[[[210, 511], [209, 511], [208, 511], [207, 511], [206, 511], [205, 511], [204, 511], [203, 511], [202, 511], [201, 511], [200, 511], [199, 511], [198, 511], [197, 511], [196, 511], [195, 511], [...",1,[['89088 1 89600 1 90111 2 90622 3 91133 4 91645 4 92156 5 92667 6 93179 6 93690 7 94202 7 94713 8 95225 8 95737 8 96249 8 96761 8 97273 8 97785 8 98297 8 98809 8 99322 7 99834 7 100346 7 100859 6...,5120,39424


In [8]:
_# add stratifiedkfold to df:
yaml_df_stratified = yaml_img_df.copy(deep=True)
strat_kfold = StratifiedKFold(shuffle = True, random_state = 42, n_splits = 8) #random_state for reproducibility, 416/8 = 52 images per fold

#split on white and non-white and add a new column fold to it:
for each_fold, (idx1,idx2) in enumerate (strat_kfold.split(X = yaml_df_stratified, y = yaml_df_stratified['source_wsi'])):
    yaml_df_stratified.loc[idx2,'fold'] = int(each_fold) #create new fold column with the fold number (up to 5)

yaml_df_stratified["fold"] = yaml_df_stratified["fold"].apply(lambda x: int(x)) # somehow doesn't turn to int, so change to int, fold from 0~4
yaml_df_stratified

,id,coordinates,source_wsi,rle,i,j,fold
0,0033bbc76b6b,"[[[[169, 228], [168, 228], [167, 228], [166, 228], [165, 228], [164, 228], [163, 228], [163, 227], [162, 227], [161, 227], [161, 226], [160, 226], [160, 225], [159, 225], [159, 224], [158, 224], [...",1,[['73387 16 73897 28 74401 38 74911 43 75420 49 75931 52 76441 55 76952 57 77462 59 77973 62 78481 68 78991 71 79502 73 80013 78 80524 82 81036 86 81548 87 82059 89 82571 90 83083 90 83595 91 8410...,10240,43008,2
1,00656c6f2690,"[[[[511, 426], [511, 426], [510, 426], [510, 425], [509, 425], [509, 424], [508, 424], [508, 423], [507, 423], [507, 422], [506, 422], [506, 421], [505, 421], [505, 420], [505, 419], [504, 419], [...",1,"[['257947 6 258457 12 258967 16 259478 18 259989 20 260500 22 261011 24 261522 26 262033 27'], ['58240 8 58750 13 59261 15 59772 16 60284 17 60795 18 61306 20 61818 20 62329 21 62841 21 63352 22 6...",10240,46080,5
2,0067d5ad2250,"[[[[318, 511], [317, 511], [316, 511], [315, 511], [314, 511], [313, 511], [312, 511], [311, 511], [310, 511], [309, 511], [308, 511], [307, 511], [306, 511], [305, 511], [304, 511], [303, 511], [...",2,[['125952 1 126463 2 126973 4 127484 5 127995 6 128506 7 129017 8 129528 9 130038 11 130549 12 131059 14 131570 15 132081 16 132592 17 133102 19 133613 20 134124 21 134635 22 135146 23 135657 24 1...,23552,22528,6
3,00d75ad65de3,"[[[[385, 393], [384, 393], [383, 393], [382, 393], [381, 393], [380, 393], [379, 393], [378, 393], [378, 392], [377, 392], [376, 392], [376, 391], [375, 391], [375, 390], [374, 390], [373, 390], [...",1,[['178854 41 178944 30 179358 58 179423 82 179862 164 180367 181 180863 199 181367 211 181869 225 182376 232 182885 240 183392 248 183899 256 184407 262 184914 273 185423 301 185932 306 186441 311...,8192,39424,2
4,00da70813521,"[[[[375, 91], [374, 91], [373, 91], [372, 91], [371, 91], [370, 91], [369, 91], [368, 91], [367, 91], [366, 91], [365, 91], [364, 91], [363, 91], [362, 91], [361, 91], [361, 90], [360, 90], [359, ...",1,[['172618 6 173128 10 173639 12 174150 14 174661 15 175173 15 175685 16 176196 17 176707 19 177219 20 177731 20 178242 22 178754 22 179265 24 179777 24 180288 26 180800 27 181311 28 181823 28 1823...,10240,46592,7
...,...,...,...,...,...,...,...
411,f86347534ec1,"[[[[421, 248], [420, 248], [419, 248], [418, 248], [417, 248], [416, 248], [415, 248], [414, 248], [414, 247], [413, 247], [412, 247], [411, 247], [411, 246], [410, 246], [410, 245], [409, 245], [...",2,[['198880 5 199391 8 199902 10 200414 10 200926 10 201438 11 201950 11 202462 12 202974 15 203487 15 203999 16 204511 17 205023 17 205535 17 206047 18 206559 19 207071 19 207583 20 208095 21 20860...,16896,20992,7
412,faba1bf818ae,"[[[[341, 157], [340, 157], [339, 157], [338, 157], [337, 157], [336, 157], [335, 157], [334, 157], [333, 157], [332, 157], [331, 157], [330, 157], [330, 156], [329, 156], [328, 156], [328, 155], [...",1,[['152181 19 152690 24 153199 28 153699 41 154140 37 154201 52 154650 49 154706 60 155161 117 155672 119 156183 120 156694 121 157204 123 157714 125 158223 128 158733 130 159240 136 159749 139 160...,3072,39424,5
413,fc6def641612,"[[[[66, 511], [65, 511], [64, 511], [63, 511], [62, 511], [61, 511], [60, 511], [59, 511], [58, 511], [57, 511], [56, 511], [55, 511], [54, 511], [53, 511], [52, 511], [51, 511], [50, 511], [49, 5...",1,[['9727 2 10236 5 10744 9 11252 13 11761 16 12271 18 12779 22 13287 26 13796 29 14306 31 14814 35 15325 36 15835 38 16345 40 16855 42 17366 43 17876 45 18384 49 18893 52 19402 55 19912 57 20422 59...,7680,40960,2
414,fd2437954fd8,"[[[[210, 511], [209, 511], [208, 511], [207, 511], [206, 511], [205, 511], [204, 511], [203, 511], [202, 511], [201, 511], [200, 511], [199, 511], [198, 511], [197, 511], [196, 511], [195, 511], [...",1,[['89088 1 89600 1 90111 2 90622 3 91133 4 91645 4 92156 5 92667 6 93179 6 93690 7 94202 7 94713 8 95225 8 95737 8 96249 8 96761 8 97273 8 97785 8 98297 8 98809 8 99322 7 99834 7 100346 7 100859 6...,5120,39424,3


In [9]:
#check if stratification worked by grouping:
grouped = yaml_df_stratified.groupby(['fold','source_wsi']) # look how it's splitted
display(grouped.fold.count())
ratio_list = []
for k in range(5):
    ratio = grouped.fold.count()[k][1]/grouped.fold.count()[k][2]
    ratio_list.append(ratio)
print("the ratios of the folds are: {}".format(ratio_list)) #ratios to check stratification

fold  source_wsi
0     1             33
      2             19
1     1             33
      2             19
2     1             33
      2             19
3     1             33
      2             19
4     1             33
      2             19
5     1             33
      2             19
6     1             33
      2             19
7     1             33
      2             19
Name: fold, dtype: int64

the ratios of the folds are: [1.736842105263158, 1.736842105263158, 1.736842105263158, 1.736842105263158, 1.736842105263158]


In [10]:
save_df = True
if save_df:
    obj_detect_bvonly_cv_train_valid_save_src = r"\\fatherserverdw\Kevin\hubmap\obj_detect_bvonly_cv_train_valid.xlsx"
    yaml_df_stratified.to_excel(obj_detect_bvonly_cv_train_valid_save_src)

In [11]:
yolov8_src = r"\\fatherserverdw\Kevin\hubmap\yolov8" # change this to wherever!
# make directory first:
train_img_src = os.path.join(yolov8_src,"train","images")
train_label_src = os.path.join(yolov8_src,"train","labels")
val_img_src = os.path.join(yolov8_src,"val","images")
val_label_src = os.path.join(yolov8_src,"val","labels")
if not os.path.exists(train_img_src):
    os.mkdir(train_img_src)
if not os.path.exists(train_label_src):
    os.mkdir(train_label_src)
if not os.path.exists(val_img_src):
    os.mkdir(val_img_src)
if not os.path.exists(val_label_src):
    os.mkdir(val_label_src)

In [12]:
### Here, let's try fold 0 = fold 0 as validation dataset and the rest as training (87.5-12.5 split since 8fold)
val_df = yaml_df_stratified[yaml_df_stratified["fold"] == 0]
val_df

,id,coordinates,source_wsi,rle,i,j,fold
12,089a9e6be240,"[[[[383, 106], [382, 106], [381, 106], [380, 106], [379, 106], [378, 106], [378, 105], [377, 105], [376, 105], [376, 104], [375, 104], [375, 103], [374, 103], [374, 102], [373, 102], [373, 101], [...",1,"[['190564 3 191075 5 191586 7 192097 9 192608 11 193119 12 193630 14 194142 14 194654 14 195166 14 195678 14 196191 13 196704 11 197217 9 197732 5'], ['114332 37 114841 44 115350 49 115860 54 1163...",2048,42496,0
30,0f5b52a768e2,"[[[[15, 508], [14, 508], [13, 508], [12, 508], [11, 508], [11, 507], [10, 507], [9, 507], [9, 506], [9, 505], [8, 505], [8, 504], [8, 503], [8, 502], [8, 501], [8, 500], [8, 499], [8, 498], [8, 49...",2,[['4593 10 5102 15 5612 17 6121 21 6631 23 7141 25 7651 27 8162 28 8673 28 9184 29 9695 28 10207 25 10718 24 11230 21 11742 20 12254 19 12765 19 13277 18 13788 18 14300 18 14810 19 15321 19 15832 ...,28160,21504,0
33,11128ba6d78c,"[[[[284, 69], [283, 69], [282, 69], [282, 68], [281, 68], [280, 68], [279, 68], [279, 67], [278, 67], [278, 66], [278, 65], [277, 65], [277, 64], [276, 64], [276, 63], [275, 63], [275, 62], [274, ...",2,[['135723 8 136232 14 136743 17 137254 19 137766 20 138277 22 138788 25 139300 26 139811 27 140323 29 140835 30 141348 30 141860 31 142373 32 142886 32 143400 30 143915 27 144430 25 144946 21 1454...,21504,21504,0
35,12ab9ac0fc55,"[[[[377, 76], [376, 76], [375, 76], [374, 76], [373, 76], [372, 76], [372, 75], [371, 75], [370, 75], [370, 74], [370, 73], [369, 73], [369, 72], [369, 71], [368, 71], [368, 70], [367, 70], [367, ...",1,[['181782 15 182291 21 182800 26 183310 29 183821 32 184332 49 184843 53 185354 56 185865 59 186376 61 186888 62 187399 64 187911 65 188423 66 188934 69 189446 71 189957 72 190469 73 190981 73 191...,6656,44544,0
36,130d4d323ce4,"[[[[110, 511], [109, 511], [108, 511], [107, 511], [106, 511], [105, 511], [104, 511], [103, 511], [102, 511], [101, 511], [100, 511], [99, 511], [98, 511], [97, 511], [96, 511], [95, 511], [94, 5...",1,[['18380 5 18891 7 19401 10 19912 12 20423 14 20934 16 21445 17 21956 20 22467 23 22979 24 23490 26 24002 26 24514 27 25026 28 25538 29 26050 30 26562 31 27074 32 27586 33 28098 34 28610 35 29123 ...,10240,39424,0
37,13aa34ced90d,"[[[[501, 353], [500, 353], [499, 353], [498, 353], [497, 353], [496, 353], [495, 353], [494, 353], [493, 353], [492, 353], [491, 353], [490, 353], [489, 353], [488, 353], [488, 352], [487, 352], [...",1,[['244561 12 245072 14 245583 16 246094 18 246604 21 247115 22 247624 26 248135 27 248645 29 249156 30 249667 31 250178 33 250689 34 251200 35 251710 37 252221 38 252732 39 253243 40 253755 40 254...,10752,45056,0
42,1a54cda8f32d,"[[[[143, 271], [142, 271], [141, 271], [140, 271], [139, 271], [138, 271], [137, 271], [136, 271], [135, 271], [135, 270], [134, 270], [133, 270], [133, 269], [132, 269], [132, 268], [131, 268], [...",1,[['35043 1 35532 2 35553 4 36025 9 36035 35 36531 57 37040 62 37550 65 38059 69 38570 71 39082 72 39595 71 40107 72 40620 71 41133 70 41645 75 42158 77 42671 83 43184 83 43697 83 44210 83 44724 81...,8704,40448,0
52,1f0d3cd4b621,"[[[[360, 225], [359, 225], [358, 225], [357, 225], [356, 225], [355, 225], [354, 225], [353, 225], [352, 225], [351, 225], [350, 225], [349, 225], [348, 225], [348, 224], [347, 224], [346, 224], [...",1,[['162494 8 163003 14 163514 16 164025 19 164537 20 165048 24 165560 25 166071 27 166583 27 167095 28 167607 29 168119 30 168630 31 169142 32 169653 33 170164 36 170674 39 171185 42 171695 45 1722...,4608,40960,0
54,212fc627af24,"[[[[372, 382], [371, 382], [370, 382], [369, 382], [368, 382], [367, 382], [367, 381], [366, 381], [366, 380], [365, 380], [365, 379], [365, 378], [365, 377], [365, 376], [365, 375], [365, 374], [...",2,[['187236 26 187746 29 188254 34 188765 35 189273 39 189783 41 190293 43 190804 44 191315 44 191827 43 192338 42 192849 42 193356 46 193867 44 194378 41 194890 36 195402 32 195914 30 196426 29 196...,21504,22528,0
63,25b98e7aea78,"[[[[240, 43], [239, 43], [238, 43], [23

In [13]:
train_df = yaml_df_stratified[yaml_df_stratified["fold"] != 0]
train_df

,id,coordinates,source_wsi,rle,i,j,fold
0,0033bbc76b6b,"[[[[169, 228], [168, 228], [167, 228], [166, 228], [165, 228], [164, 228], [163, 228], [163, 227], [162, 227], [161, 227], [161, 226], [160, 226], [160, 225], [159, 225], [159, 224], [158, 224], [...",1,[['73387 16 73897 28 74401 38 74911 43 75420 49 75931 52 76441 55 76952 57 77462 59 77973 62 78481 68 78991 71 79502 73 80013 78 80524 82 81036 86 81548 87 82059 89 82571 90 83083 90 83595 91 8410...,10240,43008,2
1,00656c6f2690,"[[[[511, 426], [511, 426], [510, 426], [510, 425], [509, 425], [509, 424], [508, 424], [508, 423], [507, 423], [507, 422], [506, 422], [506, 421], [505, 421], [505, 420], [505, 419], [504, 419], [...",1,"[['257947 6 258457 12 258967 16 259478 18 259989 20 260500 22 261011 24 261522 26 262033 27'], ['58240 8 58750 13 59261 15 59772 16 60284 17 60795 18 61306 20 61818 20 62329 21 62841 21 63352 22 6...",10240,46080,5
2,0067d5ad2250,"[[[[318, 511], [317, 511], [316, 511], [315, 511], [314, 511], [313, 511], [312, 511], [311, 511], [310, 511], [309, 511], [308, 511], [307, 511], [306, 511], [305, 511], [304, 511], [303, 511], [...",2,[['125952 1 126463 2 126973 4 127484 5 127995 6 128506 7 129017 8 129528 9 130038 11 130549 12 131059 14 131570 15 132081 16 132592 17 133102 19 133613 20 134124 21 134635 22 135146 23 135657 24 1...,23552,22528,6
3,00d75ad65de3,"[[[[385, 393], [384, 393], [383, 393], [382, 393], [381, 393], [380, 393], [379, 393], [378, 393], [378, 392], [377, 392], [376, 392], [376, 391], [375, 391], [375, 390], [374, 390], [373, 390], [...",1,[['178854 41 178944 30 179358 58 179423 82 179862 164 180367 181 180863 199 181367 211 181869 225 182376 232 182885 240 183392 248 183899 256 184407 262 184914 273 185423 301 185932 306 186441 311...,8192,39424,2
4,00da70813521,"[[[[375, 91], [374, 91], [373, 91], [372, 91], [371, 91], [370, 91], [369, 91], [368, 91], [367, 91], [366, 91], [365, 91], [364, 91], [363, 91], [362, 91], [361, 91], [361, 90], [360, 90], [359, ...",1,[['172618 6 173128 10 173639 12 174150 14 174661 15 175173 15 175685 16 176196 17 176707 19 177219 20 177731 20 178242 22 178754 22 179265 24 179777 24 180288 26 180800 27 181311 28 181823 28 1823...,10240,46592,7
...,...,...,...,...,...,...,...
411,f86347534ec1,"[[[[421, 248], [420, 248], [419, 248], [418, 248], [417, 248], [416, 248], [415, 248], [414, 248], [414, 247], [413, 247], [412, 247], [411, 247], [411, 246], [410, 246], [410, 245], [409, 245], [...",2,[['198880 5 199391 8 199902 10 200414 10 200926 10 201438 11 201950 11 202462 12 202974 15 203487 15 203999 16 204511 17 205023 17 205535 17 206047 18 206559 19 207071 19 207583 20 208095 21 20860...,16896,20992,7
412,faba1bf818ae,"[[[[341, 157], [340, 157], [339, 157], [338, 157], [337, 157], [336, 157], [335, 157], [334, 157], [333, 157], [332, 157], [331, 157], [330, 157], [330, 156], [329, 156], [328, 156], [328, 155], [...",1,[['152181 19 152690 24 153199 28 153699 41 154140 37 154201 52 154650 49 154706 60 155161 117 155672 119 156183 120 156694 121 157204 123 157714 125 158223 128 158733 130 159240 136 159749 139 160...,3072,39424,5
413,fc6def641612,"[[[[66, 511], [65, 511], [64, 511], [63, 511], [62, 511], [61, 511], [60, 511], [59, 511], [58, 511], [57, 511], [56, 511], [55, 511], [54, 511], [53, 511], [52, 511], [51, 511], [50, 511], [49, 5...",1,[['9727 2 10236 5 10744 9 11252 13 11761 16 12271 18 12779 22 13287 26 13796 29 14306 31 14814 35 15325 36 15835 38 16345 40 16855 42 17366 43 17876 45 18384 49 18893 52 19402 55 19912 57 20422 59...,7680,40960,2
414,fd2437954fd8,"[[[[210, 511], [209, 511], [208, 511], [207, 511], [206, 511], [205, 511], [204, 511], [203, 511], [202, 511], [201, 511], [200, 511], [199, 511], [198, 511], [197, 511], [196, 511], [195, 511], [...",1,[['89088 1 89600 1 90111 2 90622 3 91133 4 91645 4 92156 5 92667 6 93179 6 93690 7 94202 7 94713 8 95225 8 95737 8 96249 8 96761 8 97273 8 97785 8 98297 8 98809 8 99322 7 99834 7 100346 7 100859 6...,5120,39424,3


In [14]:
real_img_src = r"\\fatherserverdw\Kevin\hubmap\train"
val_img_list = val_df["id"]
val_img_list = [os.path.join(real_img_src,x + ".tif") for x in val_img_list]
train_img_list = train_df["id"]
train_img_list = [os.path.join(real_img_src,x + ".tif") for x in train_img_list]

In [15]:
for idx in tqdm(range(len(val_img_list))):
    val_img = val_img_list[idx]
    shutil.copy(val_img,val_img_src)

100%|██████████| 52/52 [00:08<00:00,  5.94it/s]


In [16]:
for idx in tqdm(range(len(train_img_list))):
    train_img = train_img_list[idx]
    shutil.copy(train_img,train_img_src)

100%|██████████| 364/364 [01:04<00:00,  5.68it/s]


In [17]:
### Now make a function to create .txt file for the bbox annotations for yolov8:
def create_label_txt_file(coordinates, ids, path):
    """
    :df: train_df or val_df from above
    :param ids: each ids from df above
    :param path: save path
    :return: desired yolov8 yaml file
    """
    label_txt = ''
    for coordinate in coordinates:
        label_txt += '0 '
        coor_array = np.array(coordinate[0]).astype(float)
        coor_array /= float(512)
        # transform to str
        coor_list = list(coor_array.reshape(-1).astype(str))
        coor_str = ' '.join(coor_list)
        # add string to label txt
        label_txt += f'{coor_str}\n'

    # Write labels to txt file
    with open(f'{path}/{ids}.txt', 'w') as f:
        f.write(label_txt)

In [20]:
#create file for train:
num_images = train_df.shape[0]
coordinates = []
for idx in range(num_images):
    coord = train_df.iloc[idx]["coordinates"]
    # coord = ast.literal_eval(coord)
    coordinates.append(coord)
for idx1 in tqdm(range(train_df.shape[0])):
    create_label_txt_file(coordinates = coordinates, ids = train_df.iloc[idx1]["id"], path = train_label_src)

100%|██████████| 364/364 [01:13<00:00,  4.96it/s]


In [21]:
num_images = val_df.shape[0]
coordinates = []
for idx in range(num_images):
    coord = val_df.iloc[idx]["coordinates"]
    # coord = ast.literal_eval(coord)
    coordinates.append(coord)
for idx1 in tqdm(range(val_df.shape[0])):
    create_label_txt_file(coordinates = coordinates, ids = val_df.iloc[idx1]["id"], path = val_label_src)

100%|██████████| 52/52 [00:00<00:00, 53.24it/s]
